# Regresión simbólica para sistemas de ecuaciones diferenciales

### Enrique Martínez González

[repositorio de github](https://github.com/kikeXD/symbolic_regression)

## 1 Descripción del problema

Dado un conjunto de puntos de la forma (ti, yi) se desea determinar automáticamente el sistema de ecuaciones diferenciales ordinarias que mejor describe el conjunto de datos y que además, sea lineal en los parámetros.

A continuación se describirá brevemente qué significa que la ecuación diferencial sea lineal en los parámetros y que mejor describa los datos.


### 1.1 Función lineal con respecto a los parámetros


Que el sistema sea lineal en los parámetros significa que cada componente de la parte derecha de la ecuación diferencial es una función de la forma:

    f_j(t,y) = a1 * g1(t,y) + a2 * g2(t,y) + …  + an * gn(t,y)

donde los ai son parámetros y todas las funciones gi(t,y) son funciones que dependen de la variable t, de la variable y, pero no dependen de ningún parámetro.  Un ejemplo de sistema que cumple esta características es el sistema SIR:

    S’ = - a*I*S
    I’ = a*I*S - b*I
    R’ = b*I

En este caso, se tiene que la función

    f_S (t,S,I,R) = a1 * g_s1 (t,S,I,R)

donde:

    g_s1(t,S,I,R) = -I*S.

La parte de derecha de la segunda ecuación (I’ = a\*I\*S - b*I) sería:

    f_I (t,S,I,R) = b1 * g_I1 (t,S,I,R) + b2 * g_I2 (t,S,I,R)

donde:
    
    g_I1(t,S,I,R) = I*S, y g_I2(t,S,I,R) = -I.

La parte de derecha de la tercera ecuación (R’ = b*I) sería:

    f_R (t,S,I,R) = b1 * g_R1 (t,S,I,R)

donde:
    
    g_R1(t,S,I,R) = I.

En este caso, se cumple además, que algunos de los parámetros presentes en varias de las ecuaciones son los mismos, pero eso no es un requisito para que sea lineal con respecto a las parámetros.

Este sistema SIR se puede ver gráficamente como:

![alt text](sir_model.png "Title")

y este indica en una población con una enfermedad, como se va desplazando la cantidad de susceptibles, infectados y recuperados a lo largo del tiempo.


### 1.2 Sistema de ecuaciones diferenciales que mejor describa los datos

Dado un conjunto de puntos de la forma (ti, yi) y un sistema de ecuaciones diferenciales y’ = f(t,y), se puede tener un indicador de cuán bien ese sistema describe esos datos.

Si decimos que y(ti) es la solución de la ecuación diferencial evaluada en el punto ti, entonces un indicador de cuán bien ese sistema describe los datos pudiera ser el valor L:

    L = (y(t1) - y1)^2 + (y(t2) - y2)^2 + …  + (y(tn) - yn)^2

Esto es una suma de cuadrados, que se puede escribir con una sumatoria, aunque se expresa la versión explícita.

Cuando se usa un valor como L en el que se considera la suma de cuadrados de las diferencias, se dice que estamos en presencia de un problema de mínimos cuadrados, porque lo que se quiere es minimizar esa suma de cuadrados.

Entonces, buscar el sistema de ecuaciones diferenciales que mejor describe los datos, se reduce a buscar el sistema de ecuaciones diferenciales que haga que el valor de L (la suma de cuadrados) sea lo más pequeño posible.

Pero, aquí no interesa cualquier sistema de ecuaciones diferenciales: interesa los que sean lineales con respecto a los parámetros

Este sistema de ecuaciones diferenciales deseado se vería gráficamente como un sistema que solapa al original:

![alt text](sir_model_wanted.png "Title")

### 1.3 Obteniendo los datos

De manera natural se necesitan datos correspondientes a un sistema de ecuaciones diferenciales para encontrar el sistema que los describe.

Para esto vamos a generar los datos de un modelo que ya conocemos para así comparar si el modelo obtenido por el algoritmo coincide con el original, tomaremos el modelo poblacional SIR:

    S’ = - a*I*S
    I’ = a*I*S - b*I
    R’ = b*I

Escribamos una función en python que describa este modelo:

In [1]:
# modelo del SIR
def sir_dx(X, t, a, b):
    S = X[0]
    I = X[1]
    R = X[2]

    return [-a * I * S, a * I * S - b * I, b * I]

Se recibe el tiempo dentro de la función aunque no se utilice porque estamos haciendo este ejemplo para que sea lo más abarcador y genérico posible, además resaltar que a pesar de que las EDOS normalmente se representan como f(t,y), las librerías que estamos usando de python nos piden que se coloquen de la forma utilizada en el código. 

La función de la parte derecha de la ecuación diferencial recibe también los parámetros a y b, para poder resolver la EDO para distintos valores de los parámetros. En este modelo los valores de a y b son parámetros que indican la cantidad de personas que pasan de estar saludables a infectados y de infectados a recuperados, respectivamente.

Para generar los datos debemos tener un punto inicial, o sea. Un cantidad de saludables, infectados y recuperados iniciales, además debemos conocer cuánto tiempo deseamos en los datos de entrenamiento, además de la cantidad de muestras que se le pasarán al algoritmo. Estos valores se definen en los parámetros X0, a y b.

In [2]:
import numpy as np
from scipy import integrate

def integrate_sir(time, samples, X0, a, b):
    t = np.linspace(0, time, samples)

    X, _ = integrate.odeint(sir_dx, X0, t, (a, b), full_output=True)

    S, I, R = X.T

    return (t, S, I, R)

# valores de los parámtros del modelo SIR
a = 0.3
b = 0.4

# punto inicial
X0 = [0.7, 0.3, 0]

# tiempo en el que se generarán los casos de entrenamiento
time = 20

# cantidad de muestras
n = 10000

# datos de entrenamiento
t, S, I, R = integrate_sir(time, n, X0, a, b)

La representación gráfica de estos datos es la misma mostrada con anterioridad.

## 2 Propuesta de solución

En la sección anterior definimos los términos necesarios para entender el problema que consiste en:

Dado un conjunto de puntos de la forma (ti,yi) encontrar un sistema de ecuaciones diferenciales que mejor describa los datos (en el sentido de los mínimos cuadrados).

En esta sección se tratarán varios elementos: regresión simbólica y de algoritmos genéticos.  Para usar un algoritmo genético es necesario definir cuáles son las posibles soluciones y para esas soluciones definir en qué consiste el cruzamiento, la mutación y cómo se puede saber cuán buena es una solución dada.

### 2.1 Regresión simbólica

La regresión simbólica que es un tipo de regresión que busca dentro de un espacio de expresiones matemáticas el modelo que mejor se ajuste a un conjunto de datos dados. En esta ningún modelo en particular es utilizado en el comienzo, en su lugar se generan expresiones que se forman de manera aleatoria combinando operaciones matemáticas, funciones analíticas, constantes y variables. Normalmente la regresión simbólica para funciones matemáticas es atacado con una variedad de métodos, uno de ellos es la recombinación de ecuaciones usando algoritmos evolutivos y uno de estos puede ser un algoritmo genético.

Al no requerir una especificación a priori de un modelo, la regresión simbólica no se ve afectada por el sesgo humano o las brechas desconocidas en el conocimiento del dominio. Intenta descubrir las relaciones intrínsecas del conjunto de datos, al permitir que los patrones en los propios datos revelen los modelos apropiados, en lugar de imponer una estructura de modelo que se considere matemáticamente manejable desde una perspectiva humana. La función de ajuste que impulsa la evolución de los modelos tiene en cuenta no solo las métricas de error (para garantizar que los modelos predigan los datos con precisión), sino también medidas especiales de complejidad, lo que garantiza que los modelos resultantes revelen la estructura subyacente de los datos en un manera que sea comprensible desde una perspectiva humana. Esto facilita el razonamiento y favorece las probabilidades de obtener información sobre el sistema de generación de datos.

Se ha demostrado que la regresión simbólica es un problema NP-difícil, en el sentido de que no siempre se puede encontrar la mejor expresión matemática posible para ajustarse a un conjunto de datos dado en tiempo polinomial.

### 2.2 Algoritmos genéticos

En ciencias de la computación un algoritmo genético es una metaheurística inspirada en el proceso natural de selección. Estos usualmente son utilizados para generar soluciones de alta calidad en problemas de optimización y búsqueda haciendo uso de los conocimientos de la biología, utilizando las acciones de mutación, cruzamiento y selección para modificar una población de individuos con el objetivo de obtener la solución deseada.

Para usar un algoritmo genético es necesario definir varios elementos:

- Cuáles son las posibles soluciones
- Cómo aplicar un operador de cruzamiento
- Cómo aplicar un operador de mutación
- Cómo determinar cuán buena es una solución
- Cómo determinar qué soluciones pasan a las próximas generaciones

Nuestro problema consiste en determinar el sistema de ecuaciones diferenciales que mejor se ajuste a un conjunto de datos y que sean lineales en los parámetros.

Con esto en mente, nuestras soluciones serían los posibles sistemas de ecuaciones lineales que cumplan con esas características. Por eso, en las próximas secciones vamos a describir: cómo representar las soluciones, cómo hacer cruzamientos y mutaciones, y cómo determinar cuán buena es una solución.

### 2.3 Cómo representar una solución

Como queremos resolver el problema de regresión simbólica en EDOs lineales en los parámetros, eso significa que estamos buscando una ecuación diferencial (lineal en los parámetros) que mejor ajuste los datos. Para ello, lo primero es decidir cómo representar los posibles sistemas.

Antes de empezar a dar ideas fijemos algunas ideas.

Una vez que tengamos los datos (ti, yi), tenemos algunas informaciones relevantes. Por ejemplo, si cada elemento de los datos es de la forma

    (ti, y1i, y2i, y3i)

Entonces tenemos la certeza de que el sistema tiene tres ecuaciones, y que estamos buscando una expresión de la forma:

    y1’ = f1(t, y1, y2, y3)
    y2’ = f2(t, y1, y2, y3)
    y3’ = f3(t, y1, y2, y3)

Ahora solo nos faltaría determinar cómo representar f1, f2 y f3.

Pero, a partir de los datos podemos decir que nuestras soluciones candidatas serían una lista de tres elementos, donde en la posición i está codificada la función fi(t,y1,y2,y3).

Por ejemplo, en el sistema SIR:

    S’ = - a*I*S
    I’ = a*I*S - b*I 
    R’ = b*I

La lista estaría formada por

    [-a*I*S, a*I*S - b*I, b*I]

La pregunta es entonces cómo codificar cada uno de los elementos de la función.

Se sabe que una expresión aritmética (como cada una de las funciones) se puede representar mediante un árbol, donde los nodos interiores son operadores y las hojas son variables.  Esa pudiera ser una forma de representar cada uno de los elementos: en cada posición de la lista tendríamos un árbol que representa a la parte derecha de la ecuación diferencial.

Sin embargo, no es la mejor opción posible, porque aquí nos interesan sistemas de ecuaciones diferenciales que sean lineales con respecto a los parámetros.

Eso quiere decir que cada una de las funciones fi son de la forma:

    fi (t, y1, y2, y3) = a1 * g1(t, y1, y2, y3) + a2 * g1(t, y1, y2, y3) + … + ak * gk(t, y1, y2, y3)

donde cada una de las funciones gi(t,y1,y2,y3) son funciones que no dependen de los parámetros.

Como se había planteado, en el caso del sistema SIR se tiene que:

    f_S (t,S,I,R) = a1 * g_s1 (t,S,I,R)

donde 

    g_s1(t,S,I,R) = -I*S

    f_I (t,S,I,R) = b1 * g_I1 (t,S,I,R) + b2 * g_I2 (t,S,I,R)

donde:

    g_I1(t,S,I,R) = I*S

y 
    g_I2(t,S,I,R) = -I.

    f_R (t,S,I,R) = b1 * g_R1 (t,S,I,R)
   
donde: 

    g_R1(t,S,I,R) = I.

Como cada una de las las partes derechas de la ecuación diferencial son una suma de funciones, cada una de ellas se puede representar como un diccionario con una operación especial suma, que contiene una lista, donde en la posición i de la lista estaría una operación especial de multiplicación que contendríá como hijos al parámetro ai y la función que acompaña al parámetro ai en la suma de parámetros y funciones. Estas funciones igual se pueden representar como un árbol.

Para seguir con el ejemplo del SIR, la lista asociada a la función f_S sería:

    {   
        operation: +,
        children: [
            {
                operation: *,
                children: [a1, -I*S]
            }
        ]
    }

porque es una lista formada por un único elemento.

Este árbol puede ser evaluado con el método `evaluate`. Por ejemplo si se tiene:

    f_S (t,S,I,R) = a1 * g_s1 (t,S,I,R) + a2 * g_s2 (t,S,I,R)

con 

    g_s1(t, S, I, R) = S y g_s2(t, S, I, R) = I

y 
    a1 = 3 y a2 = 4

para evaluarlo en el punto S = 5 e I = 6 obtendríamos 39 como vemos a continuación.

In [3]:
from src.utils import evaluate
from src.operation import ADD, MUL

f_S = {   
        'func': ADD['func'],
        'children': [
            {
                'func': MUL['func'],
                'children': [{'value': 3}, {"feature_name": 'S'}]
            },
            {
                'func': MUL['func'],
                'children': [{'value': 4}, {"feature_name": 'I'}]
            }
        ]
    }

evaluate(f_S, {'S': 5, 'I': 6})

39

Resaltar que los nombres utilizados en la implementación difieren con los nombres escritos en el manual, pero estos últimos son utilizados para no una explicación y mejor comprensión del código.



La lista correspondiente a la función f_I sería:

    {   
        operatios: +,
        children: [
            {
                operation: *,
                children: [a2, I*S]
            },
            {
                operation: *,
                children: [a3, -I]
            }
        ]
    }

y la lista correspondiente a la función f_R sería:

    {
        operation: +,
        children: [
            {
                operation: *,
                children: [a4, I]
            }
        ] 
    }

por lo tanto el sistema se pudiese representar como otro diccionario en donde la operación es una unión de ecuaciones, y sus hijos son los diccionarios representates de las ecuaciones antes planteados, resaltar que no se ha representando las funciones de manera de árbol, a pesar de que es la representación que se utiliza.





#### 2.3.1 Ejemplo de sistema

Ya conociendo como se representa un sistema podemos plantear un ejemplo de individuo dentro de la población del algoritmo genético, que no es más que un sistema de ecuaciones lineales con respecto a los parámetros.

    system = {
        'children': [
            # primera ecuación del sistema
            {
                'children': [
                    # primer término de la primera ecuación del sistema
                    {
                        'children': [
                            {
                                {'value': 1},
                                {'feature_name': 'I'}
                            }
                        ]
                    }
                    # segundo término de la primera ecuación del sistema
                    {
                        'children': [
                            {
                                {'value': 0.0},
                                {'feature_name': 'S'}
                            }
                        ]
                    }
                ]
            }
            # segunda ecuación del sistema
            {
                'children': [
                    # primer término de la segunda ecuación del sistema
                    {
                        'children': [
                            {
                                {'value': 2},
                                {
                                    'operation': +,
                                    'children': [
                                        {'feature_name': 'I'},
                                        {'feature_name': 'S'}
                                    ]
                                }
                            }
                        ]
                    }
                ]
            }
            # tercera ecuación del sistema
            {
                'children': [
                    # primer término de la tercera ecuación del sistema
                    {
                        'children': [
                            {
                                {'value': 3.33},
                                {'feature_name': 'S'}
                            }
                        ]
                    }
                ]
            }
        ]
    }

Esta sistema se corresponde con:

    S’ = 1 * I + 0.0 * S
    I’ = 2 * (I + S)
    R’ = 3.33 * S

Resaltar que en los diccionarios que representa el sistema, las ecuaciones y los términos de la suma en una ecuación no posee operación, porque estos son especiales.

Representaciones como estas son generadas por el método `random_system`, que son utilizadas para crear la población inicial del algoritmo. Veamos un ejemplo de la obtención de uno de estos programas y de su posterior evaluación.

In [4]:
from src.random_prog import random_system
from src.utils import render_prog

r_system = random_system(system_lenght=3,
                         operations=[ADD, MUL],
                         features_names=[['S', 'I', 'R'], ['S', 'I', 'R'], ['S', 'I', 'R']],
                         MAX_DEPTH=4)

print(render_prog(r_system))

evaluate(r_system, {'S': 5, 'I': 6, 'R': 7})

1 : 1 * (R * S) 
2 : 1 * I + 1 * (S * R) 
3 : 1 * S + 1 * (S + R) + 1 * R + 1 * S 



(35, 41, 29)

Resaltar que se inician todos los parámetros con valor 1, ya que se asume que estos serán optimizados en el futuro

### 2.4 Manipulaciones de sistemas

Como se puede ver en el sistema del ejemplo anterior aparece en la primera ecuación un parámetro igual a `0.0`, este se puede eliminar realizando un recorrido por el árbol quitando los términos que poseen parámetros menores que un `ZERO` definido. Además se puede hacer antes un recorrido redondeando los parámetros con una cantidad de cifras significativas deseada. Para esto se utilizan los métodos `filter_zero_terms_edo_system` y `round_terms_edo_system` respectivamente.

Además se puede obtener una representación explícita del sistema de ecuaciones lineales correspondiente a un árbol con el método `render_prog`

### 2.5 Cómo determinar que tan bueno es un sistema

Se define el costo de una solución como la sumatoria de la diferencia al cuadrado de los datos evaluados en el sistema analizado con los datos objtivos. Está claro que esta sumatoria mientras más pequeña sea es mejor, dado que si es 0 es que hemos encontrado un sistema que ajusta perfectamente a los datos presentes. Resaltar que a esta sumatoria se le agrega un factor de peso que este aumenta con respecto al tamaño del sistema obtenido garantizando que el sistema tienda a ser lo más pequeño posible.

El sistema presenta parámetros en cada uno de los términos de las ecuaciones, Estos valores pueden ser ajustados para mejorar el costo en dependencia del sistema, en vez de solo colocar números aleatorios. Para esto se puede crear por cada ecuación del sistema, un sistema de ecuaciones de la forma Ax=B en el que el vector de incógnitas x sean los parámetros, A sean las expresiones que acompañan a cada parámetro evaluadas en el datos analizados, y B sea el dato objetivo.

Por ejemplo, la segunda ecuación del sistema SIR es:

    f_I (t,S,I,R) = b1 * g_I1 (t,S,I,R) + b2 * g_I2 (t,S,I,R)

donde:

    g_I1(t,S,I,R) = I*S

y

    g_I2(t,S,I,R) = -I.

y supongamos que tenemos de entrada los datos

    f_I(0, 1, 2, 3) = 4
    f_I(5, 6, 7, 8) = 9

entonces se puede encontrar los valores de b1 y b2 que mejor ajusten esa ecuación con los datos planteados formando el sistema de ecuaciones

    b1 * g_I1(0, 1, 2, 3) + b2 * g_I2(0, 1, 2, 3) = 4
    b1 * g_I1(5, 6, 7, 8) + b2 * g_I2(5, 6, 7, 8) = 9

    b1 * (2 * 1) + b2 * (- 2) = 4
    b1 * (7 * 6) + b2 * (- 7) = 9

    b1 = -10/63
    b2 = -47/21

De esta forma se encuentran los mejores parámetros para cada ecuación del sistema y se obtiene finalmente un sistema de ecuaciones con los parámetros lo mejor ajustados que pueden estar dado los datos de entrada.

Esta funcionalidad la realiza el método `lineal_optimization_system`. Veamos un ejemplo en el código:

In [5]:
from src.lineal_optimization import lineal_optimization_system
from src.operation import NEG
from src.nodes import system_str, population_edo_ecuation_str

f_I = {   
        'func': ADD['func'],
        'children': [
            {
                'func': MUL['func'],
                'children': [{'value': 1}, {
                                'func': MUL['func'],
                                'children': [{'feature_name': 'I'},
                                             {"feature_name": 'S'}],
                                'format_str': MUL['format_str']
                            }],
                'format_str': MUL['format_str']
            },
            {
                'func': MUL['func'],
                'children': [{'value': 1}, {
                                'func': NEG['func'],
                                'children': [{'feature_name': 'I'}],
                                'format_str': NEG['format_str']
                            }],
                'format_str': MUL['format_str']
            }
        ],
        'format_str': population_edo_ecuation_str
    }

emtpy_equation = {'children': [], 'format_str': population_edo_ecuation_str}

#asumamos que el sistema solo tiene esa ecuación en este ejemplo
sys = {
    'children': [emtpy_equation, f_I, emtpy_equation],
    "format_str": system_str
}
print(render_prog(sys))

sys_optimized = lineal_optimization_system(system=sys, X=[{'S': 1, 'I': 2}, {'S': 6, 'I': 7}], target=[[0, 4, 0], [0, 9, 0]])

print(render_prog(sys_optimized))

1 :  
2 : (1 * (I * S)) + (1 * -(I)) 
3 :  

1 :  
2 : (-0.14285714285714274 * (I * S)) + (-2.142857142857141 * -(I)) 
3 :  



### 2.6 Mutación

Una mutación es una operación que toma un individuo y genera uno nuevo a partir de este pero con características cambiadas, aunque sigue teniendo similaridad con el individuo inicial. Para esto se realiza un recorrido aleatorio de los distintos niveles antes mencionados escogiendo uno de estos para realizar una modificación en el, esta modificación es distinta en dependencia de su altura en la representación del sistema.

Regresando al ejemplo visto con anterioridad en el sistema:

    S’ = 1 * I + 0.0 * S
    I’ = 2 * (I + S)
    R’ = 3.33 * S

Se puede realizar la mutación de eliminar el segundo sumando del primer término de la segunda ecuación. Esto resultaría en el sistema:

    S’ = 1 * I + 0.0 * S
    I’ = 2 * I
    R’ = 3.33 * S

Esta mutación fue el resultado de la eliminación de un nodo en el árbol de la expresión de un término en una de las ecuaciones, existen muchas más, como cambiar la operación de un nodo, agregar una operación, agregar un término a una ecuación, eliminar un término de una ecuación.

Se tienen varios parámetros para la elección de la probabilidad de la operación de modificación que se desea realizar. En el caso de la implementación presente a pesar de que existen funciones para mutar cada nivel del árbol, siempre se debería llamar a la función encargada de mutar el sistema, ya que esta es la que retornará el nuevo individuo ya modificado, esta es `mutate_system`.

### 2.7 Cruzamiento

Un cruzamiento es una operación que toma características de dos individuos de la población y retorna un nuevo individuo que posee caráterísticas de ambos padres. Para esto se decide primeramente que sitio se desea cruzar, cuando se plantea sitio se refiere a qué parte del sistema se desea cruzar, por ejemplo se podría cruzar a nivel de la lista de las ecuaciones, o a nivel de las expresiones que se encuentran en uno de los términos de una de las ecuaciones del sistema. Es importante que se tenga en cuenta esta organización de los individuos por niveles. En donde recordemos, la raiz es una lista de ecuaciones, el segundo nivel sería la sumatoria de términos y el tercer nivel sería las expresiones pertenecientes a cada término.

En dependencia del sitio que se desee cruzar, se toma un nodo correspondiente a cada padre y se intercambian de posición. En el caso de la implementación presente se retorna el resultado de la sustitución del nodo seleccionado en el padre 2, dentro del padre 1. Resaltar que la elección del sitio a cruzar se realiza de manera aleatoria. Si se decide cruzar en algún nivel distinto de la raiz, se toman nodos de la misma ecuación del sistema de ambos padres.

Por ejemplo, podemos tomar los sistemas:

    S’ = 1 * I + 0.0 * S
    I’ = 2 * I
    R’ = 3.33 * S

y

    S’ = 5 * R
    I’ = 3 * (R * I) + 4 * S
    R’ = 7 * I

e intercambiar las ecuaciones correspondientes a I’ resultando:

    S’ = 1 * I + 0.0 * S
    I’ = 3 * (R * I) + 4 * S
    R’ = 3.33 * S

Este resultado se puede lograr con el método `xover`

### 2.8 Cómo determinar las soluciones que pasan a la siguiente generación

Dada una población en una generación, se toma un conjunto de individuos de esta y se mutan, y se toma otro subconjunto de individuos y se cruzan entre ellos. Todos estos individuos son seleccionados de forma aleatoria. Este nuevo subconjunto formado por la población inicial de la generación, los individuos resultantes de la mutación y los individuos resultantes de los cruzamientos tienen que ser filtrados para tomar una cantidad igual a la población inicial presente en la generación con el objetivo de poder repetir este proceso múltiples veces para poder realizar numerosas generaciones. Para poder filtrar este subconjunto se toman los individuos que mejor puntuaciones obtuvieron con respecto a los datos iniciales, o sea, se toman los sistemas que mejor ajustan a los datos.

La cantidad de individuos que se mutarán y cruzarán se pueden modificar utilizando los parámetros `XOVER_SIZE` y `MUTATION_SIZE` en el método `symbolic_regression`